In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd
import itertools
from plotnine import *
import glob
from pandas.api.types import CategoricalDtype
from PIL import Image
import math

In [2]:
def get_tolerances(errors, tols = np.arange(0,1,1/1000)):
    errors = np.array(errors)
    errorRates = []
    for tol in tols:
        toAdd = np.sum(errors > tol) * 1.0 / len(errors)
        errorRates.append(toAdd)
    return errorRates

In [3]:
def average_tols(system):
    path = f'{system}'
    tol_dict = {2: [], 3: [], 4: [], 5: []}
    for file in os.listdir(path):
        with open(f'{path}/{file}', 'rb') as f:
            tol_dict[len(file.split('_')) - 2].append(get_tolerances(pickle.load(f)))
    return [np.mean(tol_dict[2], axis=0), np.mean(tol_dict[3], axis=0), np.mean(tol_dict[4], axis=0), np.mean(tol_dict[5], axis=0)] 

In [8]:
def make_df(time):
    data = {}
    data['Instrument'] = ['Duo'] * 4 + ['Trio'] * 4 + ['Quartet'] * 4 + ['Quintet'] * 4
    data['System'] = ['DTW-Chroma', 'SA-Chroma', 'SA-CQT', 'SA-BCQT'] * 4

    data['Error'] = []
    tolerances = {'DTW-Chroma': [], 'SA-Chroma': [], 'SA-CQT': [], 'SA-BCQT': []}
    for system in tolerances.keys():
        tolerances[system] = average_tols(system)
            
    for i in range(4):
        data['Error'].append(tolerances['DTW-Chroma'][i][time]*100)
        data['Error'].append(tolerances['SA-Chroma'][i][time]*100)
        data['Error'].append(tolerances['SA-CQT'][i][time]*100)
        data['Error'].append(tolerances['SA-BCQT'][i][time]*100)

    df = pd.DataFrame.from_dict(data)
    instrument_categories = CategoricalDtype(categories=["Duo", "Trio", "Quartet", "Quintet"], ordered=True)
    df.Instrument = df.Instrument.astype(instrument_categories)
    system_categories = CategoricalDtype(categories=['DTW-Chroma', 'SA-Chroma', 'SA-CQT', 'SA-BCQT'], ordered=True)
    df.System = df.System.astype(system_categories)
    
    return df

In [9]:
df = make_df(200)
colors = ["#00BA38",  "#f8766d", "#619CFF", "#ffa500"]
ms100_df = make_df(100)
ms400_df = make_df(400)

In [10]:
(ggplot(df, aes(x="System", y="Error", fill="System")) +
    geom_bar(width = 0.8, position=position_dodge2(preserve='single', width=1), stat='identity') +
    scale_y_continuous(expand = [0, 0], limits = [0, 85]) +
    scale_fill_manual(values=colors) +
    geom_crossbar(ms100_df, aes(ymin="Error", ymax="Error")) +
    geom_crossbar(ms400_df, aes(ymin="Error", ymax="Error")) +
    facet_grid('. ~ Instrument') +
    theme_bw() + 
    labs(y = "Error Rate") +
    theme(dpi=300, legend_position=(0.5, 0.03), legend_direction="horizontal", legend_title_align='bottom', 
            legend_background=element_blank(),
            legend_title = element_text(size=10),
            strip_background = element_rect(fill="#ebebeb"),
            plot_title = element_text(size=14, colour='black', margin={'b': 12}),
            axis_text_x = element_blank(),
            axis_ticks_major_x = element_blank(),
            axis_text_y = element_text(size = 12, colour='black'), 
            axis_title_x = element_blank(),
            axis_title_y = element_text(size = 12.0, margin={'r': 6.0})) +
    guides(fill=guide_legend(title="System:", title_position='left', label_position="right", override_aes = {'size': 0}))).save('./URMP.png')



/home/ibukey/anaconda3/envs/mir/lib/python3.7/site-packages/plotnine/ggplot.py:721: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/ibukey/anaconda3/envs/mir/lib/python3.7/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: ./URMPTest-seg.png


In [1]:
Image.open('URMP.png')